# Batch Analysis for hawk/dove simulation with multiple risk attitudes, no adjustment

- What % of time do agents play Hawk, by risk attitude? risk-inclined (R=1) and risk-avoidant (R8) agents play Hawk?
- Cumulative wealth analysis by risk attitude

To track how often agents play Hawk, we need to collect data for every round.

This data was generated by running:
```console
simulatingrisk/hawkdovemulti/batch_run.py --params no_adjustment --agent-data --collect-data every_round
```

Each row in the data file represents one round for each agent.

In [2]:
import polars as pl


# df = pd.read_csv("../../data/hawkdovemulti/2025-07-22T170057_747737_agent.csv")
# df = pd.read_csv("../../data/hawkdovemulti/2025-07-23T135510_859267_agent.csv")
# which batch run data to use; use variable to ensure we use matching agent and model data
batch_run_date = "2025-07-24T120337_924060"

# load agent data, drop unneeded columns, and add numeric value 1 for played hawk, 0 for played dove

df = (
    pl.read_csv(f"../../data/hawkdovemulti/{batch_run_date}_agent.csv")
        .drop("risk_level_changed")  # drop risk_level_changed; not relevant here (no adjustment = no changes)
        .rename({'risk_level': 'risk_attitude'})  # code still uses risk_level internally; relabel as risk attitude
        .with_columns(
            # add a numeric field to turn choice of play to 1/0 hawk, for aggregation            
            played_hawk=pl.when(pl.col("choice").eq("hawk")).then(1).otherwise(0)
        )
)
df.head()

RunId,iteration,Step,AgentID,risk_attitude,choice,points,played_hawk
i64,i64,i64,i64,i64,str,i64,i32
1,1,1,0,3,"""dove""",12,0
1,1,1,1,8,"""dove""",12,0
1,1,1,2,8,"""hawk""",12,1
1,1,1,3,4,"""hawk""",18,1
1,1,1,4,8,"""hawk""",15,1


## Percent of the time agents play Hawk, by risk attitude

What % of time do risk-inclined (R=1) and risk-avoidant (R8) agents play Hawk?

- Guess from observation is is >90% for R=1, <10% for R8, but we want to have statistics for this: for X trials, how many of them does R1 play Hawk more than 90% of the time?
- Also useful to have statistics e.g. R=2 played Hawk between 80-90% of the time, or whatever the result is.


In [3]:
# each row in the data frame is a play by an agent on the grid
# group by risk level, then:
# - count the number of rows (= total number of plays)
# - sum the played_hawk field (= number of times played hawk)
# - calculate percent of turns played hawk

hawk_by_risk_attitude = (
    df.group_by("risk_attitude")
        .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

hawk_by_risk_attitude

risk_attitude,n_plays,n_plays_hawk,pct_plays_hawk
i64,u32,i32,f64
9,13146745,178159,1.355157
3,15198831,10693764,70.359122
0,13280811,13101490,98.649774
6,15209556,4506272,29.6279
7,13058019,1725094,13.210993
1,12726041,11838227,93.023643
4,17567250,10583899,60.247899
8,12692401,882273,6.951191
2,13098799,11371104,86.810279


In [4]:
# output core fields as nicely styled table

(hawk_by_risk_attitude
    .select("risk_attitude", "pct_plays_hawk")
    .sort("risk_attitude")
    .rename({"risk_attitude": "Risk Attitude", "pct_plays_hawk": "% plays Hawk"})
    .style.tab_header(title="% of time agents play Hawk by Risk Attitude")
    .fmt_number("% plays Hawk", decimals=1)
)

GT(_tbl_data=shape: (10, 2)
┌───────────────┬──────────────┐
│ Risk Attitude ┆ % plays Hawk │
│ ---           ┆ ---          │
│ i64           ┆ f64          │
╞═══════════════╪══════════════╡
│ 0             ┆ 98.649774    │
│ 1             ┆ 93.023643    │
│ 2             ┆ 86.810279    │
│ 3             ┆ 70.359122    │
│ 4             ┆ 60.247899    │
│ 5             ┆ 39.645375    │
│ 6             ┆ 29.6279      │
│ 7             ┆ 13.210993    │
│ 8             ┆ 6.951191     │
│ 9             ┆ 1.355157     │
└───────────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x326570b30>, _boxhead=Boxhead([ColInfo(var='Risk Attitude', type=<ColInfoTypeEnum.default: 1>, column_label='Risk Attitude', column_align='right', column_width=None), ColInfo(var='% plays Hawk', type=<ColInfoTypeEnum.default: 1>, column_label='% plays Hawk', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10b131e80>, _spanners=Spanners([]), _heading=Heading(title='% of time agents play Hawk by Risk Attitude', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x325e48560>, _formats=[<great_tables._gt_data.FormatInfo object at 0x3259fc1a0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', v

In [5]:
import altair as alt

alt.Chart(hawk_by_risk_attitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).properties(title="% of time agents play Hawk")

alt.Chart(...)

Context: how many runs are these numbers drawn from?



In [6]:
total_unique_runs = len(df["RunId"].unique())
total_iterations = len(df["iteration"].unique())
n_combinations = int(total_unique_runs / total_iterations)

# Step is the round count indicator
longest_run = df["Step"].max()    # highest across all runs
# average of max value for each run. Group by run, get max Step, average —> returns a dataframe, get first Step value
average_run = df.group_by("RunId").agg(pl.col("Step").max()).mean()["Step"].first()

print(f"""{total_unique_runs:,} total unique runs; {total_iterations} iterations of {n_combinations} different parameter combinations.

Longest run: {longest_run} steps
Average run: {average_run:.1f} steps
""")


13,500 total unique runs; 100 iterations of 135 different parameter combinations.

Longest run: 109 steps
Average run: 44.9 steps



### Analysis filtered by simulation parameters

In [7]:
# identify the last round of each run
# for both wealth analysis and model parameters, we want to look at the last round (Step) of each run

last_round_df = df.group_by("RunId").agg(pl.col("Step").max())
last_round_df.head()

RunId,Step
i64,i64
8789,57
8146,31
12996,31
9983,31
11421,31


In [8]:
# load model data and filter to last round for each run
full_model_df = pl.read_csv(f"../../data/hawkdovemulti/{batch_run_date}_model.csv")
model_df = last_round_df.join(full_model_df, on=['RunId', 'Step'], how="left")
# limit to only those fields that are needed for our analysis
model_df = model_df.select("RunId", "iteration", "risk_distribution", "play_neighborhood", "observed_neighborhood", "grid_size")
model_df.head()

RunId,iteration,risk_distribution,play_neighborhood,observed_neighborhood,grid_size
i64,i64,str,i64,i64,i64
8789,89,"""skewed right""",8,4,5
8146,46,"""skewed right""",8,8,5
12996,96,"""bimodal""",4,24,5
9983,83,"""skewed right""",4,8,5
11421,21,"""bimodal""",8,4,5


In [9]:
print(f"""Simulation parameters:

Grid size: {', '.join(str(n) for n in sorted(model_df["grid_size"].unique()))}
Iniital risk distribution: {', '.join(str(n) for n in sorted(model_df["risk_distribution"].unique()))}
Play neighborhood size: {', '.join(str(n) for n in sorted(model_df["play_neighborhood"].unique()))}
Observed neighborhood sized: {', '.join(str(n) for n in sorted(model_df["observed_neighborhood"].unique()))}

""")

Simulation parameters:

Grid size: 5, 10, 25
Iniital risk distribution: bimodal, normal, skewed left, skewed right, uniform
Play neighborhood size: 4, 8, 24
Observed neighborhood sized: 4, 8, 24




In [10]:
# join agent data with model data so we can filter by starting parameters
agent_df_params = df.join(model_df, on=["RunId", "iteration"], how="left")

#### Grid size


In [11]:
hawk_by_gridsize_riskattitude = (
    agent_df_params.group_by("grid_size", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_gridsize_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(column=alt.Column("grid_size", title="Grid Size")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

#### Play neighborhood


In [12]:
hawk_by_playnhood_riskattitude = (
    agent_df_params.group_by("play_neighborhood", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_playnhood_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(column=alt.Column("play_neighborhood", title="Play Neighborhood")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

#### Observed neighborhood


In [13]:
hawk_by_obsnhood_riskattitude = (
    agent_df_params.group_by("observed_neighborhood", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_obsnhood_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(column=alt.Column("observed_neighborhood", title="Observed Neighborhood")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

#### Initial risk distribution


In [14]:
hawk_by_initialdist_riskattitude = (
    agent_df_params.group_by("risk_distribution", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_initialdist_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(facet=alt.Facet("risk_distribution", title="Initial Risk Distribution"), columns=3
).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

## Cumulative Wealth analysis


- mean and quartiles for wealth by R
- does mean vary between Rs or is it roughly the same?
  - quartiles look different, but need a statistic; esp. compare lower-R quartile to higher-R quartile
     - expect/hope that lower quartile is higher for R1 than R8, higher quartile is higher for R8 than R1
     - what's going on in the middle?


-----

Because our simulations run for different lengths before stopping, and because our simulations include a range of different play neighborhoods (which affects total payoff), we scale points (wealth) by play neighborhood and simulation run length for comparison across simulations.


In [15]:
# combine the last round dataframe and full agent dataframe to get just the last round
agents_last_round_df = (
    last_round_df.join(df, on=['RunId', 'Step'], how="left")
        # join on model parameters, for filtering and scaling by play neighborhood
        .join(model_df, on=["RunId", "iteration"])
        .with_columns(
            # calculate a scaled points value so we can compare across runs with different length and play neighborhood
            scaled_points=pl.col("points").truediv(pl.col("play_neighborhood")).truediv(pl.col("Step")).mul(100)
        )
    
)

# hard to compare points across runs, since it depends on how long the simulation ran; scale points by number of runs
# agents_last_round_df['scaled_points'] = agents_last_round_df.apply(lambda x: (x['points'] / x['Step'])*10, axis=1)

# merge with model parameters, for filtering
# agents_last_round_df = pd.merge(agents_last_round_df, model_df, on=["RunId", "iteration"])

agents_last_round_df.head(10)

RunId,Step,iteration,AgentID,risk_attitude,choice,points,played_hawk,risk_distribution,play_neighborhood,observed_neighborhood,grid_size,scaled_points
i64,i64,i64,i64,i64,str,i64,i32,str,i64,i64,i64,f64
8789,57,89,0,2,"""hawk""",687,1,"""skewed right""",8,4,5,150.657895
8789,57,89,1,7,"""dove""",689,0,"""skewed right""",8,4,5,151.096491
8789,57,89,2,2,"""hawk""",771,1,"""skewed right""",8,4,5,169.078947
8789,57,89,3,7,"""dove""",608,0,"""skewed right""",8,4,5,133.333333
8789,57,89,4,9,"""dove""",687,0,"""skewed right""",8,4,5,150.657895
8789,57,89,5,8,"""dove""",714,0,"""skewed right""",8,4,5,156.578947
8789,57,89,6,5,"""hawk""",628,1,"""skewed right""",8,4,5,137.719298
8789,57,89,7,7,"""dove""",686,0,"""skewed right""",8,4,5,150.438596
8789,57,89,8,3,"""dove""",603,0,"""skewed right""",8,4,5,132.236842


First plot unscaled wealth distribution by risk attitude.

In [16]:
# our data has a lot of rows; enable vegafusion so altair can calculate quartiles for us
alt.data_transformers.enable("vegafusion")

alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
   x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("points", title="Wealth "),
).properties(title="Cumulative Wealth (unscaled) by Risk Attitude")

alt.Chart(...)

Now plot scaled wealth distribution by risk attitude.

In [17]:
wealthchart_title = alt.TitleParams(
    "Cumulative Wealth by Risk Attitude",
    subtitle=["Wealth scaled by simulation length and play neighborhood"]
)

alt.Chart(agents_last_round_df, title=wealthchart_title).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth "),
) 

alt.Chart(...)

In [18]:
# Wanted to try a violin plot for comparison, but Altair/Vega can't handle the scale of our data,
# and I couldn't find an example of how to precompute the values to feed into an Altair mark area chart.


For comparison's sake, plot wealth at round 31 across all simulations, scaled only by play neighborhood.

This distribution looks similar to the scaled wealth plot.    

In [19]:
# what if we look at wealth at round 31 across simulations?

agents_round31_df = df.filter(pl.col("Step").eq(31)).join(model_df, on=["RunId", "iteration"]).with_columns(
    scaled_points=pl.col("points").truediv(pl.col("play_neighborhood")).mul(8)
)

alt.Chart(agents_round31_df).mark_boxplot(extent="min-max").encode(
   x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth"),
).properties(title="Cumulative wealth at round 31 across runs")

alt.Chart(...)

Calculate quartiles and other statistics, and output as a table for reference.

Calculate quartiles and other statistics and output as a table, for reference.

In [20]:
# Altair boxplot is calculating quartiles for us, but we can calculate them directly as well

wealth_by_risk_attitude = agents_last_round_df.group_by("risk_attitude").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("risk_attitude")

In [21]:
# output wealth by risk attitude as nicely styled table

(wealth_by_risk_attitude
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude", subtitle="Wealth scaled by simulation length and play neighborhood")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (10, 7)
┌───────────────┬───────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ Risk Attitude ┆ min       ┆ max        ┆ mean       ┆ Q1         ┆ Q2         ┆ Q3         │
│ ---           ┆ ---       ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ i64           ┆ f64       ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞═══════════════╪═══════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ 0             ┆ 0.0       ┆ 300.0      ┆ 148.685253 ┆ 114.919355 ┆ 149.691358 ┆ 174.596774 │
│ 1             ┆ 0.0       ┆ 300.0      ┆ 144.020105 ┆ 113.206215 ┆ 147.580645 ┆ 167.79661  │
│ 2             ┆ 0.0       ┆ 300.0      ┆ 145.429545 ┆ 112.5      ┆ 147.580645 ┆ 170.564516 │
│ 3             ┆ 0.0       ┆ 300.0      ┆ 143.420712 ┆ 110.849057 ┆ 137.5      ┆ 165.957447 │
│ 4             ┆ 0.0       ┆ 300.0      ┆ 141.715225 ┆ 109.946237 ┆ 134.659091 ┆ 157.258065 │
│ 5             ┆ 1.694915  ┆ 300.0      ┆ 139.42156  ┆ 114.344262 ┆ 137.5      ┆ 150.806452 │
│ 6             ┆ 43.220339 ┆ 300.0      ┆ 143.347455 ┆ 125.403226 ┆ 143.025362 ┆ 154.301075 │
│ 7             ┆ 64.705882 ┆ 300.0      ┆ 147.048369 ┆ 137.037037 ┆ 149.327957 ┆ 158.064516 │
│ 8             ┆ 87.096774 ┆ 300.0      ┆ 151.041036 ┆ 140.254237 ┆ 150.403226 ┆ 161.693548 │
│ 9             ┆ 96.774194 ┆ 203.225806 ┆ 150.287365 ┆ 141.397849 ┆ 150.0      ┆ 160.887097 │
└───────────────┴───────────┴────────────┴────────────┴────────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x32713e360>, _boxhead=Boxhead([ColInfo(var='Risk Attitude', type=<ColInfoTypeEnum.default: 1>, column_label='Risk Attitude', column_align='right', column_width=None), ColInfo(var='min', type=<ColInfoTypeEnum.default: 1>, column_label='min', column_align='right', column_width=None), ColInfo(var='max', type=<ColInfoTypeEnum.default: 1>, column_label='max', column_align='right', column_width=None), ColInfo(var='mean', type=<ColInfoTypeEnum.default: 1>, column_label='mean', column_align='right', column_width=None), ColInfo(var='Q1', type=<ColInfoTypeEnum.default: 1>, column_label='Q1', column_align='right', column_width=None), ColInfo(var='Q2', type=<ColInfoTypeEnum.default: 1>, column_label='Q2', column_align='right', column_width=None), ColInfo(var='Q3', type=<ColInfoTypeEnum.default: 1>, column_label='Q3', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x327a7ad50>, _spanners=Spanners([]), _heading=Heading(title='Cumulative Wealth by Risk Attitude', subtitle='Wealth scaled by simulation length and play neighborhood', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x327d9d2e0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x327d9cad0>, <great_tables._gt_data.FormatInfo object at 0x327adeb10>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=Opti

### Analysis filtered by simulation parameters

How does the wealth distribution vary based on other starting parameters?

#### Grid size

In [22]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth"),
).facet(column=alt.Column("grid_size", title="Grid Size")).properties(title=wealthchart_title)

alt.FacetChart(...)

In [83]:
wealthchart_title

TitleParams({
  subtitle: ['Wealth scaled by simulation length and play neighborhood'],
  text: 'Cumulative Wealth by Risk Attitude'
})

In [159]:
# calculate mean & quartiles for risk attitude by grid size
wealth_by_risk_grid = agents_last_round_df.group_by("risk_attitude", "grid_size").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("grid_size", "risk_attitude")


selection = alt.selection_point(fields=['grid_size'], bind='legend')

wealth_mean_grid = alt.Chart(wealth_by_risk_grid).mark_line(interpolate="monotone").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.Y("mean", title="Wealth (mean)").scale(zero=False),
    color=alt.Color("grid_size:N", title="Grid Size"),
    opacity=alt.when(selection).then(alt.value(1.0)).otherwise(alt.value(0.4))
).add_params(
    selection
)

wealth_spread_grid = alt.Chart(wealth_by_risk_grid).mark_area(interpolate="monotone").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.Y("Q3").scale(zero=False),
    y2="Q1",
    color=alt.Color("grid_size:N", title="Grid Size"),
    opacity=alt.when(selection).then(alt.value(0.3)).otherwise(alt.value(0.1))
).add_params(
    selection
)

# combine the charts for multiple ways to view
grid_wealth_title = wealthchart_title.copy()
grid_wealth_title['text'] += " and Grid Size — Mean and Quartiles"

(wealth_mean_grid | wealth_spread_grid | (wealth_mean_grid + wealth_spread_grid)
).resolve_legend(color="shared").properties(title=grid_wealth_title)

alt.HConcatChart(...)

Click on items in the Grid Size legend to change the chart opacity to focus on a particular group.

In [160]:
# turn  grid size into label for context in the table
(wealth_by_risk_grid.with_columns(grid_size=pl.lit("Grid Size: ").add(pl.col("grid_size").cast(pl.datatypes.String)))
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude and Grid Size", subtitle="Wealth scaled by simulation length and play neighborhood")
     .tab_stub(rowname_col="Risk Attitude", groupname_col="grid_size")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (30, 8)
┌────────────┬────────────┬───────────┬────────────┬───────────┬───────────┬───────────┬───────────┐
│ Risk       ┆ grid_size  ┆ min       ┆ max        ┆ mean      ┆ Q1        ┆ Q2        ┆ Q3        │
│ Attitude   ┆ ---        ┆ ---       ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ ---        ┆ str        ┆ f64       ┆ f64        ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
│ i64        ┆            ┆           ┆            ┆           ┆           ┆           ┆           │
╞════════════╪════════════╪═══════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 0          ┆ Grid Size: ┆ 0.0       ┆ 300.0      ┆ 148.22116 ┆ 125.0     ┆ 150.0     ┆ 161.96236 │
│            ┆ 5          ┆           ┆            ┆ 5         ┆           ┆           ┆ 6         │
│ 1          ┆ Grid Size: ┆ 0.0       ┆ 300.0      ┆ 142.61666 ┆ 113.15789 ┆ 148.61111 ┆ 161.55914 │
│            ┆ 5          ┆           ┆            ┆           ┆ 5         ┆ 1         ┆           │
│ 2          ┆ Grid Size: ┆ 0.0       ┆ 300.0      ┆ 143.50073 ┆ 109.44444 ┆ 147.61904 ┆ 167.86723 │
│            ┆ 5          ┆           ┆            ┆ 9         ┆ 4         ┆ 8         ┆ 2         │
│ 3          ┆ Grid Size: ┆ 1.415094  ┆ 300.0      ┆ 140.98151 ┆ 104.43548 ┆ 136.96236 ┆ 162.09677 │
│            ┆ 5          ┆           ┆            ┆ 2         ┆ 4         ┆ 6         ┆ 4         │
│ 4          ┆ Grid Size: ┆ 2.419355  ┆ 300.0      ┆ 139.74679 ┆ 103.22580 ┆ 133.02238 ┆ 156.85028 │
│            ┆ 5          ┆           ┆            ┆ 4         ┆ 6         ┆ 8         ┆ 2         │
│ …          ┆ …          ┆ …         ┆ …          ┆ …         ┆ …         ┆ …         ┆ …         │
│ 5          ┆ Grid Size: ┆ 1.694915  ┆ 300.0      ┆ 139.60384 ┆ 115.10416 ┆ 137.5     ┆ 150.80645 │
│            ┆ 25         ┆           ┆            ┆           ┆ 7         ┆           ┆ 2         │
│ 6          ┆ Grid Size: ┆ 43.220339 ┆ 300.0      ┆ 143.56589 ┆ 125.45180 ┆ 143.14516 ┆ 154.43548 │
│            ┆ 25         ┆           ┆            ┆ 8         ┆ 7         ┆ 1         ┆ 4         │
│ 7          ┆ Grid Size: ┆ 64.705882 ┆ 300.0      ┆ 147.19194 ┆ 137.09677 ┆ 149.38524 ┆ 158.06451 │
│            ┆ 25         ┆           ┆            ┆ 5         ┆ 4         ┆ 6         ┆ 6         │
│ 8          ┆ Grid Size: ┆ 87.096774 ┆ 300.0      ┆ 151.13959 ┆ 140.32258 ┆ 150.42372 ┆ 161.69354 │
│            ┆ 25         ┆           ┆            ┆ 5         ┆ 1         ┆ 9         ┆ 8         │
│ 9          ┆ Grid Size: ┆ 96.774194 ┆ 203.225806 ┆ 150.37525 ┆ 141.26344 ┆ 150.06944 ┆ 160.88709 │
│            ┆ 25         ┆           ┆            ┆ 2         ┆ 1         ┆ 4         ┆ 7         │
└────────────┴────────────┴───────────┴────────────┴───────────┴───────────┴───────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x670bcb080>, _boxhead=Boxhead([ColInfo(var='Risk Attitude', type=<ColInfoTypeEnum.stub: 2>, column_label='Risk Attitude', column_align='right', column_width=None), ColInfo(var='grid_size', type=<ColInfoTypeEnum.row_group: 3>, column_label='grid_size', column_align='left', column_width=None), ColInfo(var='min', type=<ColInfoTypeEnum.default: 1>, column_label='min', column_align='right', column_width=None), ColInfo(var='max', type=<ColInfoTypeEnum.default: 1>, column_label='max', column_align='right', column_width=None), ColInfo(var='mean', type=<ColInfoTypeEnum.default: 1>, column_label='mean', column_align='right', column_width=None), ColInfo(var='Q1', type=<ColInfoTypeEnum.default: 1>, column_label='Q1', column_align='right', column_width=None), ColInfo(var='Q2', type=<ColInfoTypeEnum.default: 1>, column_label='Q2', column_align='right', column_width=None), ColInfo(var='Q3', type=<ColInfoTypeEnum.default: 1>, column_label='Q3', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x673c22c00>, _spanners=Spanners([]), _heading=Heading(title='Cumul

#### Play neighborhood

In [115]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth"),
).facet(column=alt.Column("play_neighborhood", title="Play Neighborhood")
).properties(title=wealthchart_title)

alt.FacetChart(...)

In [141]:
# calculate mean & quartiles for risk attitude by play neighborhood
wealth_by_risk_playnhood = agents_last_round_df.group_by("risk_attitude", "play_neighborhood").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("play_neighborhood", "risk_attitude")


playn_selection = alt.selection_point(fields=['play_neighborhood'], bind='legend')


wealth_mean_playnhood = alt.Chart(wealth_by_risk_playnhood).mark_line(interpolate="monotone").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.Y("mean", title="Wealth (mean)").scale(zero=False),
    color=alt.Color("play_neighborhood:N", title="Play Neighborhood"),
    opacity=alt.when(playn_selection).then(alt.value(1.0)).otherwise(alt.value(0.4))
).add_params(
    playn_selection
)

wealth_spread_playnhood = alt.Chart(wealth_by_risk_playnhood).mark_area(interpolate="monotone").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.Y("Q3").scale(zero=False),
    y2="Q1",
    color=alt.Color("play_neighborhood:N", title="Play Neighborhood"),
    opacity=alt.when(playn_selection).then(alt.value(0.3)).otherwise(alt.value(0.1))
).add_params(
    playn_selection
)

playn_wealth_title = wealthchart_title.copy()
playn_wealth_title['text'] += " and Play Neighborhood — Mean and Quartiles"

# combine the charts for multiple ways to view
(wealth_mean_playnhood | wealth_spread_playnhood | (wealth_mean_playnhood + wealth_spread_playnhood)
).resolve_legend(color="shared").properties(title=playn_wealth_title)

alt.HConcatChart(...)

Click on items in the Play Neighborhood legend to change the chart opacity to focus on a particular group.

In [161]:
(wealth_by_risk_playnhood.with_columns(play_neighborhood=pl.lit("Play Neighborhood: ").add(pl.col("play_neighborhood").cast(pl.datatypes.String)))
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude and Play Neighborhood", subtitle="Wealth scaled by simulation length and play neighborhood")
     .tab_stub(rowname_col="Risk Attitude", groupname_col="play_neighborhood")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (30, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ Risk       ┆ play_neigh ┆ min        ┆ max       ┆ mean      ┆ Q1        ┆ Q2        ┆ Q3        │
│ Attitude   ┆ borhood    ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ ---        ┆ ---        ┆ f64        ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
│ i64        ┆ str        ┆            ┆           ┆           ┆           ┆           ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 0          ┆ Play Neigh ┆ 0.0        ┆ 300.0     ┆ 151.70060 ┆ 79.83871  ┆ 150.0     ┆ 220.16129 │
│            ┆ borhood: 4 ┆            ┆           ┆ 2         ┆           ┆           ┆           │
│ 1          ┆ Play Neigh ┆ 0.0        ┆ 300.0     ┆ 148.56029 ┆ 88.709677 ┆ 149.51923 ┆ 191.10169 │
│            ┆ borhood: 4 ┆            ┆           ┆ 5         ┆           ┆ 1         ┆ 5         │
│ 2          ┆ Play Neigh ┆ 0.0        ┆ 300.0     ┆ 150.64837 ┆ 99.193548 ┆ 149.19354 ┆ 212.09677 │
│            ┆ borhood: 4 ┆            ┆           ┆ 5         ┆           ┆ 8         ┆ 4         │
│ 3          ┆ Play Neigh ┆ 0.0        ┆ 300.0     ┆ 147.68716 ┆ 101.81818 ┆ 129.50819 ┆ 186.57407 │
│            ┆ borhood: 4 ┆            ┆           ┆ 3         ┆ 2         ┆ 7         ┆ 4         │
│ 4          ┆ Play Neigh ┆ 0.0        ┆ 300.0     ┆ 145.39259 ┆ 102.08333 ┆ 125.51020 ┆ 175.0     │
│            ┆ borhood: 4 ┆            ┆           ┆ 2         ┆ 3         ┆ 4         ┆           │
│ …          ┆ …          ┆ …          ┆ …         ┆ …         ┆ …         ┆ …         ┆ …         │
│ 5          ┆ Play Neigh ┆ 82.598039  ┆ 266.52542 ┆ 139.92863 ┆ 125.51020 ┆ 141.82098 ┆ 151.34408 │
│            ┆ borhood:   ┆            ┆ 4         ┆ 7         ┆ 4         ┆ 8         ┆ 6         │
│            ┆ 24         ┆            ┆           ┆           ┆           ┆           ┆           │
│ 6          ┆ Play Neigh ┆ 82.748538  ┆ 256.98924 ┆ 143.76249 ┆ 134.95762 ┆ 145.96774 ┆ 154.16666 │
│            ┆ borhood:   ┆            ┆ 7         ┆ 6         ┆ 7         ┆ 2         ┆ 7         │
│            ┆ 24         ┆            ┆           ┆           ┆           ┆           ┆           │
│ 7          ┆ Play Neigh ┆ 90.607345  ┆ 244.75806 ┆ 148.30579 ┆ 142.06989 ┆ 150.0     ┆ 156.98924 │
│            ┆ borhood:   ┆            ┆ 5         ┆ 4         ┆ 2         ┆           ┆ 7         │
│            ┆ 24         ┆            ┆           ┆           ┆           ┆           ┆           │
│ 8          ┆ Play Neigh ┆ 95.127119  ┆ 247.17741 ┆ 151.94756 ┆ 145.96774 ┆ 153.22580 ┆ 158.60215 │
│            ┆ borhood:   ┆            ┆ 9         ┆ 2         ┆ 2         ┆ 6         ┆ 1         │
│            ┆ 24         ┆            ┆           ┆           ┆           ┆           ┆           │
│ 9          ┆ Play Neigh ┆ 113.306452 ┆ 189.91935 ┆ 151.20844 ┆ 145.69892 ┆ 150.80645 ┆ 157.79569 │
│            ┆ borhood:   ┆            ┆ 5         ┆ 8         ┆ 5         ┆ 2         ┆ 9         │
│            ┆ 24         ┆            ┆           ┆           ┆           ┆           ┆           │
└────────────┴────────────┴────────────┴───────────┴───────────┴───────────┴───────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x673c21a30>, _boxhead=Boxhead([ColInfo(var='Risk Attitude', type=<ColInfoTypeEnum.stub: 2>, column_label='Risk Attitude', column_align='right', column_width=None), ColInfo(var='play_neighborhood', type=<ColInfoTypeEnum.row_group: 3>, column_label='play_neighborhood', column_align='left', column_width=None), ColInfo(var='min', type=<ColInfoTypeEnum.default: 1>, column_label='min', column_align='right', column_width=None), ColInfo(var='max', type=<ColInfoTypeEnum.default: 1>, column_label='max', column_align='right', column_width=None), ColInfo(var='mean', type=<ColInfoTypeEnum.default: 1

#### Observed neighborhood

In [118]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth"),
).facet(column=alt.Column("observed_neighborhood", title="Observed Neighborhood")
).properties(title=wealthchart_title)

alt.FacetChart(...)

In [143]:
# calculate mean & quartiles for risk attitude by play neighborhood
wealth_by_risk_obsvnhood = agents_last_round_df.group_by("risk_attitude", "observed_neighborhood").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("observed_neighborhood", "risk_attitude")


obsvn_selection = alt.selection_point(fields=['observed_neighborhood'], bind='legend')


wealth_mean_obsvnhood = alt.Chart(wealth_by_risk_obsvnhood).mark_line(interpolate="monotone").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.Y("mean", title="Wealth (mean)").scale(zero=False),
    color=alt.Color("observed_neighborhood:N", title="Observed Neighborhood"),
    opacity=alt.when(obsvn_selection).then(alt.value(1.0)).otherwise(alt.value(0.4))
).add_params(
    obsvn_selection
)

wealth_spread_obsvnhood = alt.Chart(wealth_by_risk_obsvnhood).mark_area(interpolate="monotone").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.Y("Q3").scale(zero=False),
    y2="Q1",
    color=alt.Color("observed_neighborhood:N", title="Observed Neighborhood"),
    opacity=alt.when(obsvn_selection).then(alt.value(0.3)).otherwise(alt.value(0.1))
).add_params(
    obsvn_selection
)

obsvn_wealth_title = wealthchart_title.copy()
obsvn_wealth_title['text'] += " and Observed Neighborhood — Mean and Quartiles"

# combine the charts for multiple ways to view
(wealth_mean_obsvnhood | wealth_spread_obsvnhood | (wealth_mean_obsvnhood + wealth_spread_obsvnhood)
).resolve_legend(color="shared").properties(title=obsvn_wealth_title)

alt.HConcatChart(...)

Click on items in the Observed Neighborhood legend to change the chart opacity to focus on a particular group.

In [162]:
(wealth_by_risk_obsvnhood.with_columns(observed_neighborhood=pl.lit("Observed Neighborhood: ").add(pl.col("observed_neighborhood").cast(pl.datatypes.String)))
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude and Observed Neighborhood", subtitle="Wealth scaled by simulation length and play neighborhood")
     .tab_stub(rowname_col="Risk Attitude", groupname_col="observed_neighborhood")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (30, 8)
┌────────────┬────────────┬───────────┬────────────┬───────────┬───────────┬───────────┬───────────┐
│ Risk       ┆ observed_n ┆ min       ┆ max        ┆ mean      ┆ Q1        ┆ Q2        ┆ Q3        │
│ Attitude   ┆ eighborhoo ┆ ---       ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ ---        ┆ d          ┆ f64       ┆ f64        ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
│ i64        ┆ ---        ┆           ┆            ┆           ┆           ┆           ┆           │
│            ┆ str        ┆           ┆            ┆           ┆           ┆           ┆           │
╞════════════╪════════════╪═══════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 0          ┆ Observed   ┆ 0.0       ┆ 300.0      ┆ 153.05584 ┆ 125.54644 ┆ 150.0     ┆ 183.87096 │
│            ┆ Neighborho ┆           ┆            ┆ 3         ┆ 8         ┆           ┆ 8         │
│            ┆ od: 4      ┆           ┆            ┆           ┆           ┆           ┆           │
│ 1          ┆ Observed   ┆ 38.709677 ┆ 300.0      ┆ 150.96171 ┆ 118.20652 ┆ 148.63505 ┆ 174.32795 │
│            ┆ Neighborho ┆           ┆            ┆ 3         ┆ 2         ┆ 7         ┆ 7         │
│            ┆ od: 4      ┆           ┆            ┆           ┆           ┆           ┆           │
│ 2          ┆ Observed   ┆ 38.709677 ┆ 300.0      ┆ 154.9343  ┆ 124.57627 ┆ 149.59677 ┆ 183.06451 │
│            ┆ Neighborho ┆           ┆            ┆           ┆ 1         ┆ 4         ┆ 6         │
│            ┆ od: 4      ┆           ┆            ┆           ┆           ┆           ┆           │
│ 3          ┆ Observed   ┆ 67.741935 ┆ 300.0      ┆ 152.79100 ┆ 124.19354 ┆ 145.29569 ┆ 168.51092 │
│            ┆ Neighborho ┆           ┆            ┆ 2         ┆ 8         ┆ 9         ┆ 9         │
│            ┆ od: 4      ┆           ┆            ┆           ┆           ┆           ┆           │
│ 4          ┆ Observed   ┆ 64.386792 ┆ 300.0      ┆ 154.57354 ┆ 124.86559 ┆ 147.69230 ┆ 169.75806 │
│            ┆ Neighborho ┆           ┆            ┆ 4         ┆ 1         ┆ 8         ┆ 5         │
│            ┆ od: 4      ┆           ┆            ┆           ┆           ┆           ┆           │
│ …          ┆ …          ┆ …         ┆ …          ┆ …         ┆ …         ┆ …         ┆ …         │
│ 5          ┆ Observed   ┆ 1.694915  ┆ 300.0      ┆ 131.15903 ┆ 104.34782 ┆ 125.80645 ┆ 149.46236 │
│            ┆ Neighborho ┆           ┆            ┆ 2         ┆ 6         ┆ 2         ┆ 6         │
│            ┆ od: 24     ┆           ┆            ┆           ┆           ┆           ┆           │
│ 6          ┆ Observed   ┆ 43.220339 ┆ 298.5      ┆ 136.96537 ┆ 111.82795 ┆ 138.20754 ┆ 152.41935 │
│            ┆ Neighborho ┆           ┆            ┆ 9         ┆ 7         ┆ 7         ┆ 5         │
│            ┆ od: 24     ┆           ┆            ┆           ┆           ┆           ┆           │
│ 7          ┆ Observed   ┆ 64.705882 ┆ 291.129032 ┆ 145.00685 ┆ 131.45161 ┆ 149.19354 ┆ 160.33653 │
│            ┆ Neighborho ┆           ┆            ┆           ┆ 3         ┆ 8         ┆ 8         │
│            ┆ od: 24     ┆           ┆            ┆           ┆           ┆           ┆           │
│ 8          ┆ Observed   ┆ 87.711864 ┆ 247.033898 ┆ 151.55643 ┆ 141.93548 ┆ 151.34408 ┆ 162.5     │
│            ┆ Neighborho ┆           ┆            ┆ 4         ┆ 4         ┆ 6         ┆           │
│            ┆ od: 24     ┆           ┆            ┆           ┆           ┆           ┆           │
│ 9          ┆ Observed   ┆ 96.774194 ┆ 203.225806 ┆ 151.46619 ┆ 141.93548 ┆ 150.40322 ┆ 162.09677 │
│            ┆ Neighborho ┆           ┆            ┆ 8         ┆ 4         ┆ 6         ┆ 4         │
│            ┆ od: 24     ┆           ┆            ┆           ┆           ┆           ┆           │
└────────────┴────────────┴───────────┴────────────┴───────────┴───────────┴───────────┴───────────┘, _body=<great_tables._gt_data.Bod

#### Initial risk distribution

In [117]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth"),
).facet(facet=alt.Facet("risk_distribution", title="Initial Risk Distribution"), columns=3
).properties(title=wealthchart_title)

alt.FacetChart(...)

In [130]:
# calculate mean & quartiles for risk attitude by play neighborhood
wealth_by_risk_dist = agents_last_round_df.group_by("risk_attitude", "risk_distribution").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("risk_distribution", "risk_attitude")

riskdist_selection = alt.selection_point(fields=['risk_distribution'], bind='legend')

wealth_mean_riskdist = alt.Chart(wealth_by_risk_dist).mark_line(interpolate="monotone").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.Y("mean", title="Wealth (mean)").scale(zero=False),
    color=alt.Color("risk_distribution:N", title="Distribution"),
    opacity=alt.when(riskdist_selection).then(alt.value(1.0)).otherwise(alt.value(0.4))
).add_params(
    riskdist_selection
)

wealth_spread_riskdist = alt.Chart(wealth_by_risk_dist).mark_area(interpolate="monotone").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.Y("Q3").scale(zero=False),
    y2="Q1",
    color=alt.Color("risk_distribution:N", title="Distribution"),
    opacity=alt.when(riskdist_selection).then(alt.value(0.3)).otherwise(alt.value(0.1))
).add_params(
    riskdist_selection
)

riskdist_wealth_title = wealthchart_title.copy()
riskdist_wealth_title['text'] += " and Initial Risk Distribution — Mean and Quartiles"

# combine the charts for multiple ways to view
(wealth_mean_riskdist | wealth_spread_riskdist | (wealth_mean_riskdist + wealth_spread_riskdist)
).resolve_legend(color="shared").properties(title=riskdist_wealth_title)

alt.HConcatChart(...)

Click on items in the Distribution legend to change the chart opacity to focus on a particular group.

In [163]:
# output values as a nice table so we can reference them if needed
(wealth_by_risk_dist.with_columns(risk_distribution=pl.lit("Initial Risk Distribution: ").add(pl.col("risk_distribution")))
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude and Initial Risk Distribution", subtitle="Wealth scaled by simulation length and play neighborhood")
     .tab_stub(rowname_col="Risk Attitude", groupname_col="risk_distribution")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (50, 8)
┌────────────┬────────────┬───────────┬────────────┬───────────┬───────────┬───────────┬───────────┐
│ Risk       ┆ risk_distr ┆ min       ┆ max        ┆ mean      ┆ Q1        ┆ Q2        ┆ Q3        │
│ Attitude   ┆ ibution    ┆ ---       ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ ---        ┆ ---        ┆ f64       ┆ f64        ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
│ i64        ┆ str        ┆           ┆            ┆           ┆           ┆           ┆           │
╞════════════╪════════════╪═══════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 0          ┆ Initial    ┆ 0.0       ┆ 300.0      ┆ 152.12405 ┆ 124.77272 ┆ 150.0     ┆ 185.48387 │
│            ┆ Risk Distr ┆           ┆            ┆ 1         ┆ 7         ┆           ┆ 1         │
│            ┆ ibution:   ┆           ┆            ┆           ┆           ┆           ┆           │
│            ┆ bim…       ┆           ┆            ┆           ┆           ┆           ┆           │
│ 1          ┆ Initial    ┆ 0.0       ┆ 300.0      ┆ 153.14757 ┆ 125.20833 ┆ 150.0     ┆ 185.48387 │
│            ┆ Risk Distr ┆           ┆            ┆ 4         ┆ 3         ┆           ┆ 1         │
│            ┆ ibution:   ┆           ┆            ┆           ┆           ┆           ┆           │
│            ┆ bim…       ┆           ┆            ┆           ┆           ┆           ┆           │
│ 2          ┆ Initial    ┆ 0.0       ┆ 300.0      ┆ 153.28330 ┆ 125.40322 ┆ 150.0     ┆ 185.48387 │
│            ┆ Risk Distr ┆           ┆            ┆ 2         ┆ 6         ┆           ┆ 1         │
│            ┆ ibution:   ┆           ┆            ┆           ┆           ┆           ┆           │
│            ┆ bim…       ┆           ┆            ┆           ┆           ┆           ┆           │
│ 3          ┆ Initial    ┆ 0.0       ┆ 300.0      ┆ 155.97132 ┆ 126.20967 ┆ 150.0     ┆ 185.08064 │
│            ┆ Risk Distr ┆           ┆            ┆ 2         ┆ 7         ┆           ┆ 5         │
│            ┆ ibution:   ┆           ┆            ┆           ┆           ┆           ┆           │
│            ┆ bim…       ┆           ┆            ┆           ┆           ┆           ┆           │
│ 4          ┆ Initial    ┆ 1.363636  ┆ 300.0      ┆ 158.67531 ┆ 137.09677 ┆ 150.0     ┆ 184.67741 │
│            ┆ Risk Distr ┆           ┆            ┆           ┆ 4         ┆           ┆ 9         │
│            ┆ ibution:   ┆           ┆            ┆           ┆           ┆           ┆           │
│            ┆ bim…       ┆           ┆            ┆           ┆           ┆           ┆           │
│ …          ┆ …          ┆ …         ┆ …          ┆ …         ┆ …         ┆ …         ┆ …         │
│ 5          ┆ Initial    ┆ 4.032258  ┆ 300.0      ┆ 150.64528 ┆ 136.96236 ┆ 148.38709 ┆ 156.48148 │
│            ┆ Risk Distr ┆           ┆            ┆ 8         ┆ 6         ┆ 7         ┆ 1         │
│            ┆ ibution:   ┆           ┆            ┆           ┆           ┆           ┆           │
│            ┆ uni…       ┆           ┆            ┆           ┆           ┆           ┆           │
│ 6          ┆ Initial    ┆ 56.451613 ┆ 300.0      ┆ 149.44544 ┆ 137.90322 ┆ 148.79032 ┆ 154.83871 │
│            ┆ Risk Distr ┆           ┆            ┆ 2         ┆ 6         ┆ 3         ┆           │
│            ┆ ibution:   ┆           ┆            ┆           ┆           ┆           ┆           │
│            ┆ uni…       ┆           ┆            ┆           ┆           ┆           ┆           │
│ 7          ┆ Initial    ┆ 84.677419 ┆ 300.0      ┆ 147.83073 ┆ 138.30645 ┆ 149.19354 ┆ 154.03225 │
│            ┆ Risk Distr ┆           ┆            ┆ 4         ┆ 2         ┆ 8         ┆ 8         │
│            ┆ ibution:   ┆           ┆            ┆           ┆           ┆           ┆           │
│            ┆ uni…       ┆           ┆            ┆           ┆           ┆           ┆           │
│ 8          ┆ Initial    ┆ 93.14